In [1]:
import pandas as pd, numpy, scipy, matplotlib

nacitanie dat do dafatfamov

In [5]:
df_product = pd.read_csv("product.csv")
df_session = pd.read_csv("session.csv")
df_users = pd.read_csv("users.csv")

,code\tproduct_ean\tstore_name\tlocation
0,IN\t3550122019246\tTeghra\tAsia/Kolkata
1,JP\t3293910072053\tToba\tAsia/Tokyo
2,IN\t8879380535301\tKundla\tAsia/Kolkata
3,CL\t5488010366760\tChiguayante\tAmerica/Santiago
4,PG\t3161153029229\tMount Hagen\tPacific/Port_M...
...,...
14903,PK\t3231680675975\tJhol\tAsia/Karachi
14904,MX\t6658350634617\tContla\tAmerica/Mexico_City
14905,BD\t0108089628792\tManikchari\tAsia/Dhaka
14906,DO\t4035395311840\tSanta Cruz de Barahona\tAme...
